In [4]:
import glob
import os
import onnxruntime
import numpy as np
from PIL import Image
import torchvision.transforms as transforms

path_of_the_model = '' #the path to the onnx file of your model
path_of_folder_where_your_images_are = '' #path to the folder where the images you want the predict are

# Load the ONNX model
onnx_model_path = path_of_the_model
ort_session = onnxruntime.InferenceSession(onnx_model_path)
image_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.Grayscale(num_output_channels=3),  # Convert to RGB
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

image_folder_path = path_of_folder_where_your_images_are
#image_folder_path = "K:/Docs/stage_ingenieur/models/datas/cxrs/pathological"
# List all image files in the folder
image_files = glob.glob(os.path.join(image_folder_path, "*.JPEG"))

# Loop through each image and perform inference
for image_file in image_files:
    # Preprocess the image
    image = Image.open(image_file)
    try:
        input_data = image_transform(image).unsqueeze(0).numpy()
    except Exception as e:
        print(f"Skipping image {image_file} due to error: {e}")
        continue
    
    # Run inference
    ort_inputs = {ort_session.get_inputs()[0].name: input_data}
    ort_outputs = ort_session.run(None, ort_inputs)
    
    # Post-process results (if needed)
    # You might need to interpret the model's output to get meaningful predictions
    output = ort_outputs[0]
    class_probabilities = np.exp(output) / np.sum(np.exp(output), axis=1, keepdims=True)

    # Get predicted class index
    predicted_class_index = np.argmax(class_probabilities)

    # Print results

    # Print predictions or save them as needed
    print(f"Image: {image_file}, Predictions: {ort_outputs}")
    print("Predicted class index:", predicted_class_index)
    print("Class probabilities:", class_probabilities)